<a href="https://colab.research.google.com/github/renanpyd/INVESTMENTS/blob/main/EscolhaAtivosInvest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install yfinance

In [38]:
from datetime import datetime #para trabalhar com datas
from bs4 import BeautifulSoup #coletar dados de páginas web
from requests.models import Response # Resposta
from requests.sessions import Request #Requisição HTTPS
import pandas as pd #para trabalhar com tabelas
import yfinance as yf #para coletar dados financeiros
import requests #para enviar e receber solicitações da internet
import matplotlib.pyplot as plt #para trabalhar com gráficos

In [ ]:
url = '<https://www.fundsexplorer.com.br/ranking>'

agent = {"User-Agent":"Mozilla/5.0"}
response = requests.get(url, headers = agent)

soup = BeautifulSoup(response.text, 'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table), decimal=',', thousands='.')[0]
stocks_df = df
print(df)

In [ ]:
symbols = stocks_df['Códigodo fundo'] + '.SA'
symbols

In [ ]:
start = datetime(2020, 3, 30)
end = datetime.today() #hoje

def get_stocks(name):
  stock = yf.download(name, start=start, end=end)
  return stock

tickers = [tick for tick in symbols]

stock_dict = {}

for tick in tickers:
    try:
        stock_dict[tick] = get_stocks(tick)
    except:
        print('Exception occured for -  ', tick)

In [ ]:
data = yf.download('FIVN11.SA', start = '2020-3-30', end = datetime.today())
data

In [ ]:
discount_dict = {}

def calculate_discount(df):
    max_price = df['Adj Close'].describe()['75%']
    min_price = df['Adj Close'].min()
    return (min_price - max_price)/max_price
    
for tick in stock_dict.keys():
    discount_dict[tick] = calculate_discount(stock_dict[tick])

In [ ]:
discount_dict = {}

def calculate_discount(df):
    max_price = df['Adj Close'].describe()['75%']
    min_price = df['Adj Close'].min()
    return (min_price - max_price)/max_price
    
for tick in stock_dict.keys():
    discount_dict[tick] = calculate_discount(stock_dict[tick])

In [ ]:
returns = pd.DataFrame()

for key in stock_dict.keys():
    returns[key] = stock_dict[key]['Adj Close'].pct_change()
    
returns.head()

In [ ]:
def get_stockname(ticker):
    return [x for x in stocks_df[stocks_df['Códigodo fundo'] == ticker]['Códigodo fundo']][0]

std_list = []

for tick in stock_info_df['ticker']:
    std_list.append(returns[tick].std()) # risk metric
    
stock_info_df['risk'] = std_list
    
stock_info_df

In [ ]:
def get_sector(stock):
    return [x for x in stocks_df[symbols == stock]['Setor']]

stock_info_df = pd.DataFrame.from_dict(discount_dict, orient='index').reset_index()
stock_info_df.columns = ['ticker', 'discount']
stock_info_df['sector'] = [get_sector(tick)[0] for tick in stock_info_df['ticker']]

stock_info_df

In [ ]:
import plotly.express as px
stock_info_df = stock_info_df.dropna()

fig = px.scatter(stock_info_df, x = "risk", y = "discount", size = "current_price", color = "sector", hover_name = "ticker", log_x = True)
fig.show()